In [ ]:
from pathlib import Path

import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import pdr
from marslab.imgops.masking import skymask, threshold_mask

from roi_classifier import (
    add_rois,
    apply_homography,
    apply_kmeans_to_masked,
    average_spectra,
    compute_homography,
    filter_connected_components,
    load_data,
    LoadResult,
    mask_cube,
    plot_spectra,
    SHARED_BANDS
)
from pdr.pds4_tools.extern.zscale import zscale

In [ ]:
# pipeline settings

SMALL_ROI_PARAMS = {
    "min_region_sz": 1200,
    "max_region_sz": 5000,
    "center_thresh_dif": 0,
    "density_threshold": 0.7
}
FLOAT_MIN_MAX_AREA = (800, 20000)
LARGE_ROI_PARAMS = {
    "min_region_sz": 500,
    "max_region_sz":  5000,
    "center_thresh_dif": 50,
    "density_threshold": 0.5
}
N_CLUSTERS = {"float": 3, "large": 4, "small": 4}
THRESHOLD_KWARGS = {
    'percentiles': (6, 99),
    'operator': 'and'
}
SKYMASK_KWARGS = {
    'percentile': 75,
    'edge_params': {"maximum": 5, "erosion": 3},
    'input_median': 5,
    'trace_maximum': 5,
    'cutoffs': {
        "extent": 0.05, "coverage": None, "v": 0.9, "h": None
    },
    'input_mask_dilation': None,
    'input_stretch': (10, 1),
    'floodfill': True,
    'trim_params': {"trim": False},
    'clear': True,
    'colorblock': False,
    'respect_mask': False,
}

### Load, Pre-process cube

In [ ]:
# NOTE: get_zcam_bandset expects a directory structure like:
# sol
# sol/iof
# sol/pix_map

SEARCH_PATH = Path("/datascratch/zcam_data/products/1164/iof")
SEQ_ID, OBS_IX = None, 2
# NOTE: load_data crops the images inline; trim_margins() is no longer needed
load_result: LoadResult = load_data(
    SEARCH_PATH, seq_id=SEQ_ID, obs_ix=OBS_IX, do_apply_pixmaps=True
) 

In [ ]:
l_cube, r_cube = load_result["l_cube"], load_result["r_cube"]

In [ ]:
# b/c we might have applied NaN values to bands of the cube, always use
# the original bands for computing homography (NaNs make cv2's SIFT
# implementation unhappy)

base_bands = load_result["base_bands"]
h_matrix = compute_homography(
    base_bands[SHARED_BANDS["L"]], base_bands[SHARED_BANDS["R"]],
)
l_cube_mapped = apply_homography(l_cube, h_matrix, r_cube[0].shape)

In [ ]:
plt.imshow(l_cube_mapped[0])

In [ ]:
full_cube = []
shared_band_index = (
    sorted(load_result["bandset"].raw)
     .index(SHARED_BANDS['L'])
)
# average left/right views of Bayer and 800nm bands
for band in range(shared_band_index + 1):
    img = (l_cube_mapped[band] + r_cube[band]) / 2
    full_cube.append(img)

# add left bands
l_num_bands = l_cube.shape[0]
for band in range(shared_band_index + 1, l_num_bands):
    full_cube.append(l_cube[band])
# add right bands
r_num_bands = r_cube.shape[0]
for band in range(shared_band_index + 1, r_num_bands):
    full_cube.append(r_cube[band])

full_cube = np.array(full_cube)

In [ ]:
full_cube.shape

### Masking shaddows and sky

In [ ]:
# NOTE: this is an unnecessary copy if we didn't apply pixmaps
r_cube_base = np.array(
    [a for b, a in base_bands.items() if b.startswith('R')]
)
# mask shadows
shadow_tmask = threshold_mask(r_cube_base, **THRESHOLD_KWARGS)

# mask the sky
sky_tmask = skymask(r_cube_base, **SKYMASK_KWARGS)

In [ ]:
# mask the overlap bewteen the cameras
overlap_mask = (l_cube_mapped[shared_band_index] == 0)

# combine the masks into one
feature_mask = np.logical_or(shadow_tmask, sky_tmask)
full_tmask = np.logical_or(feature_mask, overlap_mask)

#### NOTE 
I cut mask_bad_pixels() b/c I think it may be too
redundant and unstable with pixmap masking added to shadow
and sky masking. (the low end is redundant already!)

In [ ]:
cube_preprocessed = mask_cube(full_cube, full_tmask)
del r_cube_base

cube_preprocessed.mask = (
    cube_preprocessed.mask | ~np.isfinite(cube_preprocessed)
)
### Identify small surfaces
float_clustered = apply_kmeans_to_masked(cube_preprocessed,
                                            N_CLUSTERS["float"])
float_rock_mask = filter_connected_components(float_clustered,
                                              *FLOAT_MIN_MAX_AREA)

In [ ]:
plt.imshow(cube_preprocessed[12])

In [ ]:
plt.imshow(float_clustered, cmap='tab10', interpolation='nearest')

In [ ]:
plt.imshow(float_rock_mask)

### Create separate masks for large and small surfaces

In [ ]:
# compute cube prioritizing large surfaces
l_surfs_mask = np.logical_or(float_rock_mask, full_tmask)
l_surfs = mask_cube(cube_preprocessed, l_surfs_mask)

# compute cube prioritizing small surfaces
s_surfs_mask = np.logical_or(~float_rock_mask, full_tmask)
s_surfs = mask_cube(cube_preprocessed, s_surfs_mask)

In [ ]:
plt.imshow(l_surfs[0])

In [ ]:
plt.imshow(s_surfs[0])

### Cluster large/small surface masks

In [ ]:
l_clustered = apply_kmeans_to_masked(l_surfs, N_CLUSTERS["large"])
s_clustered = apply_kmeans_to_masked(s_surfs, N_CLUSTERS["small"])

In [ ]:
plt.imshow(l_clustered, cmap='tab20', interpolation='nearest')

In [ ]:
plt.imshow(s_clustered, cmap='tab10', interpolation='nearest')

### Identify ROI in classifications

In [ ]:
s_rois = add_rois(s_clustered, **SMALL_ROI_PARAMS)
l_rois = add_rois(l_clustered, **LARGE_ROI_PARAMS)

In [ ]:
from marslab.imgops.imgutils import enhance_color

In [ ]:
enhance_color

In [ ]:
# creates rgb image from right bayer bands
red = r_cube[0]
green = r_cube[1]
blue = r_cube[2]

rgb = np.ma.masked_invalid(np.stack([red, green, blue], axis=-1))
rgb_normalized = enhance_color(rgb, (0, 1), 0.1)

plt.imshow(rgb_normalized)

In [ ]:
solar_elevation

In [ ]:
# filter rois based on spectral error and similarity

DO_APPLY_R_STAR = True

# NOTE: using Data.metaget() here can be sketchy
#  b/c ZCAM metadata has multiple solar elevation values
#  in different coordinate systems, not always in the same
#  order in different label versions!
if DO_APPLY_R_STAR is True:
    meta = load_result["bandset"].metadata
    incidence = meta["INCIDENCE_ANGLE"].unique().mean()
    photometric_scaling = np.cos(incidence * 2 * np.pi / 360)
else:
    photometric_scaling = 1

if DO_APPLY_R_STAR is False:
    y_axis_units = "IOF"
else:
    y_axis_units = "Relative Reflectance"

# scale the cube
photometrically_calibrated = cube_preprocessed / photometric_scaling

In [ ]:
rois = s_rois + l_rois

# convert rect coords to plot coords
plt_rois = []
for i in range(len(rois)):
    x1, y1, w, h = rois[i]
    x2 = x1 + w
    y2 = y1 + h
    plt_rois.append((x1, y1, x2, y2))

# calculate average spectra in roi regions
spectra, stds = average_spectra(photometrically_calibrated, plt_rois)

In [ ]:
avg_err = stds.mean(axis=1)

In [ ]:
# testing using error as cutoff parameter
# might delete...
err_threshold = 0.05

roi_to_omit = []
for i, err in enumerate(avg_err):
    if err > err_threshold:
        roi_to_omit.append(i)

In [ ]:
# sanity check
print(roi_to_omit)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))
im = ax.imshow(rgb_normalized)

COLORS = [
        "#ebac23",
        "#D80039",
        "#008cf9",
        "#AB358A",
        "#F5029F",
        "#0E00EE",
        "#00EB39",
        "#f05a28",
        "#15E4B1",
        "#D16174",
        "#136B8C",
        "#DD6E12",
        "#834C71"
    ]

color_i = 0
for i, (x1, y1, x2, y2) in enumerate(rois):
    if i in roi_to_omit:
        continue
    
    if color_i == len(COLORS):
        color_i = 0
    
    curr_color = COLORS[color_i]
    
    roi = patches.Rectangle((x1, y1), x2, y2, edgecolor=curr_color, facecolor='none', linewidth=2)
    ax.add_patch(roi)
    
    color_i+=1

plt.show()

In [ ]:
markers = [
    'o',
    's',
    '^',
    'v',
    '*',
    'D',
    'H'
]

In [ ]:
plot_spectra(spectra, stds, COLORS, markers)

### testing...

In [ ]:
fits = pdr.read("C:\\Users\\Lars\\Downloads\\roi_SOL1376_zcam04056_RSM1458.fits.gz")

In [ ]:
fits

In [ ]:
fits['GREEN LEFT']

In [ ]:
plt.imshow(fits['GREEN LEFT'])